In [1]:
import numpy as np
from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.table import Column
from numpy import *
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.wcs import WCS
from astropy.io import fits
from matplotlib.colors import LogNorm
from astropy.utils.data import download_file
import warnings

catalog_path='/Users/EvanB.LAPTOP-8CHCIOMA/Downloads/'

#A100-NSA Catalog
myTab=Table.read(catalog_path+'a100-nsa-for-adap.fits')

In [24]:
myTab['ZDIST'][2]

0.0034598312

In [2]:
# R-band Magnitude Limit
Mrmax = -18.2
Mr = myTab['ABSMAG'][:,4]
Mrflag = Mr < Mrmax

# Velocity input
vinput = int(input()) # Asks for a velocity to create a new cut based off of
v = myTab['Vhelio'] # Recessional velocity from catalog
vflag = (v > -1*vinput) & (v < vinput) # Applies a cut (+/- the input velocity) to the data set 

cut = Mrflag & vflag # Combines both cuts

cuttab = myTab[cut]

 1500


# Computing 5th Nearest Neighbor

In [3]:
ra = np.array(cuttab['RA']) # Right ascension of each galaxy
dec = np.array(cuttab['DEC']) # Declination of each galaxy

points = []
for i in range(len(dec)):
    points.append((ra[i],dec[i])) # Array of tuples of ra and dec (each galaxy now represented by a point)

def distance(ra1, dec1, ra2, dec2):
    return ((ra1 - ra2) ** 2 + (dec1 - dec2) ** 2) ** ( 1 / 2) # Distance between two points    

def Five_Nearest(points, target, K):
    pts = []
    n = len(points)
    d = []
 
    for i in range(n):
        d.append({
            "first": distance(points[i][0], points[i][1], target[0], target[1]), # Finds distance between target point and another point
            "second": i # Stores new point that had its distance relative to the target found
        })
     
    d = sorted(d, key=lambda l:l["first"]) # Sorts d from smallest to largest distance for each target point
 
    for i in range(K): # Loops through for however many times are specified by K (I will use K=6)
        pt = []
        pt.append(points[d[i]["second"]][0]) # First element of a tuple in an array of nearest points sorted by distance
        pt.append(points[d[i]["second"]][1]) # Second element of a tuple in an array of nearest points sorted by distance
        pts.append(pt) # Stores all individual pt values as tuples
 
    return pts # Returns the six nearest galaxies to a target galaxy (the first of which is the original target galaxy)


In [22]:
K = 1
target = points[0]
a = Five_Nearest(points, target, K)
print(a)

[[0.8125990364294372, 16.14570390458945]]


In [17]:
# Find five nearest galaxies to each galaxy

five_nn = [] # Store nearest neighbors
K = 6 # Number of times to calaculate nearest neighbors

for i in range(len(points)):
    target = points[i] # Switches target point
    for pt in Five_Nearest(points, target, K): # 
        five_nn.append((pt[0], pt[1]))

In [5]:
print(five_nn[0])

(0.8125990364294372, 16.14570390458945)


In [6]:
# Separating target points from fifth nearest points

target_fnn = points #five_nn[0::6] # list of original target galaxies
fifth_nearest_neighbors = five_nn[5::6] # fifth nearest point of each target

print('FNN:',fifth_nearest_neighbors[0:2])
print('Targets:',target_fnn[0:2])

FNN: [(2.1247261268751685, 18.324074686728974), (5.8267271563667835, 13.70221435354031)]
Targets: [(0.8125990364294372, 16.14570390458945), (4.439792860539326, 11.450287516539865)]


In [8]:
# Finding angular distance

dist_fifthnn = []

# Calculates distance between each galaxy and its fifth nearest meighbor

for i in range(len(fifth_nearest_neighbors)):
    target0 = target_fnn[i] # Changes target on each loop through
    dist_fifthnn.append(distance(fifth_nearest_neighbors[i][0], fifth_nearest_neighbors[i][1], target0[0], target0[1])) # Produces distance array
 
angle = np.array(dist_fifthnn)
print(angle[0:2])

[2.54302512 2.64476109]


In [30]:
# Redshifts of fifth nearest neighbors

z_fnn = []
for i in range(len(fifth_nearest_neighbors)):
    for j in range(len(points)):
        if fifth_nearest_neighbors[i][0] == points[j][0]:
            z_fnn.append(cuttab['ZDIST'][j])

In [31]:
z_fnn[0]

0.019063886

In [9]:
# Converting angular distances to kpc

# Array of redshifts for each fifth nearest neighbor
cutz = cuttab['ZDIST']
z = np.array(cutz)

# Convert angles in degrees to radians
radians = (np.pi/180)*(angle)

# Finding distance, in m, between the Sun and the galaxies
H0 = 2.3*10**(-18) #/s
c = 3*10**8 #m/s

v = z*c 
d = v/H0 #m

# Using small angle approximation to convert to kpc
r_5 = (d*radians)*(3*10**(-19)) #kpc
print(r_5[0:2])
print(radians[0:2])

[6008.93638592 6639.36834382]
[0.04438416 0.04615979]


In [10]:
# Find fifth nearest neighbor density
density = 1/(r_5**2)
print(density[0:2]) # \kpc^2

[2.76952180e-08 2.26854017e-08]


## Ignore

In [ ]:
ra = cuttab['RA']
dec = cuttab['DEC']
ccode = cuttab['ZDIST']

#Print number of points
print(len(ra))

#Plot RA vs DEC and color it by recessional velocity
plt.figure(figsize=(12,5))
plt.scatter(ra,dec,c=ccode,cmap='viridis',alpha=0.8,s=2.5)
plt.xlim(reversed(plt.xlim()))
plt.title('Sample Galaxies')
plt.xlabel('RA (degrees)')
plt.ylabel('DEC (degrees)')
plt.xlim(270,100)
plt.ylim(0,70)
cbar = plt.colorbar()
cbar.set_label('Redshift',rotation=270,labelpad=20)
plt.clim(0.002,0.026)